# Model using Neural Network

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [2]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [3]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [4]:
from sklearn.utils import shuffle

shuffled_df = shuffle(df, random_state = 42)

## Split Data for Training and Validation


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state=42)

## The training and validation set size

In [6]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [7]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [8]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [9]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [10]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [11]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [12]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [13]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [14]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [15]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [16]:
x = keras.layers.Dense(64, activation = 'relu')(all_features)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [17]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [18]:
model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 3s 14ms/step - loss: 0.7278 - accuracy: 0.5159 - val_loss: 0.7012 - val_accuracy: 0.4751
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7062 - accuracy: 0.5159 - val_loss: 0.7007 - val_accuracy: 0.4972
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7079 - accuracy: 0.5007 - val_loss: 0.6999 - val_accuracy: 0.5138
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6997 - accuracy: 0.5408 - val_loss: 0.6974 - val_accuracy: 0.5138
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6897 - accuracy: 0.5256 - val_loss: 0.7006 - val_accuracy: 0.4862
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6911 - accuracy: 0.5256 - val_loss: 0.7007 - val_accuracy: 0.4917
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6875 - accuracy: 0.5726 - val_loss: 0.6972 - val_accuracy: 0.5138
Epoch 8/300


Epoch 59/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6616 - accuracy: 0.5989 - val_loss: 0.6936 - val_accuracy: 0.5414
Epoch 60/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6521 - accuracy: 0.5934 - val_loss: 0.6939 - val_accuracy: 0.5691
Epoch 61/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6524 - accuracy: 0.6238 - val_loss: 0.6937 - val_accuracy: 0.5359
Epoch 62/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6566 - accuracy: 0.5934 - val_loss: 0.6961 - val_accuracy: 0.5470
Epoch 63/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6554 - accuracy: 0.6100 - val_loss: 0.6950 - val_accuracy: 0.5414
Epoch 64/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6654 - accuracy: 0.5809 - val_loss: 0.6938 - val_accuracy: 0.5193
Epoch 65/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6582 - accuracy: 0.6100 - val_loss: 0.6958 - val_accuracy: 0.5414
Epoch 

37/37 [==============================] - 0s 3ms/step - loss: 0.6409 - accuracy: 0.6086 - val_loss: 0.7061 - val_accuracy: 0.5193
Epoch 117/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6486 - accuracy: 0.6127 - val_loss: 0.7051 - val_accuracy: 0.5249
Epoch 118/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6451 - accuracy: 0.6169 - val_loss: 0.7108 - val_accuracy: 0.5359
Epoch 119/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6374 - accuracy: 0.6196 - val_loss: 0.7112 - val_accuracy: 0.5193
Epoch 120/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6593 - accuracy: 0.6017 - val_loss: 0.7091 - val_accuracy: 0.5138
Epoch 121/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6440 - accuracy: 0.6183 - val_loss: 0.7112 - val_accuracy: 0.5193
Epoch 122/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6283 - accuracy: 0.6584 - val_loss: 0.7083 - val_accuracy: 0.5083
Epoch 123/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6273 - accuracy: 0.6252 - val_loss: 0.7075 - val_accuracy: 0.5470
Epoch 174/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6308 - accuracy: 0.6155 - val_loss: 0.7125 - val_accuracy: 0.5359
Epoch 175/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6238 - accuracy: 0.6279 - val_loss: 0.7215 - val_accuracy: 0.5028
Epoch 176/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6377 - accuracy: 0.6432 - val_loss: 0.7079 - val_accuracy: 0.5083
Epoch 177/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6183 - accuracy: 0.6667 - val_loss: 0.7147 - val_accuracy: 0.4972
Epoch 178/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6335 - accuracy: 0.6141 - val_loss: 0.7150 - val_accuracy: 0.4972
Epoch 179/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6371 - accuracy: 0.6196 - val_loss: 0.7184 - val_accuracy: 0.5138
Epoch 180/300

37/37 [==============================] - 0s 3ms/step - loss: 0.6176 - accuracy: 0.6224 - val_loss: 0.7250 - val_accuracy: 0.5414
Epoch 231/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6236 - accuracy: 0.6362 - val_loss: 0.7223 - val_accuracy: 0.5249
Epoch 232/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6137 - accuracy: 0.6501 - val_loss: 0.7268 - val_accuracy: 0.5028
Epoch 233/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6275 - accuracy: 0.6100 - val_loss: 0.7284 - val_accuracy: 0.5028
Epoch 234/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6127 - accuracy: 0.6501 - val_loss: 0.7300 - val_accuracy: 0.5304
Epoch 235/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6257 - accuracy: 0.6252 - val_loss: 0.7268 - val_accuracy: 0.5249
Epoch 236/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6272 - accuracy: 0.6418 - val_loss: 0.7210 - val_accuracy: 0.5414
Epoch 237/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5953 - accuracy: 0.6584 - val_loss: 0.7435 - val_accuracy: 0.5138
Epoch 288/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5889 - accuracy: 0.6501 - val_loss: 0.7494 - val_accuracy: 0.5249
Epoch 289/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6667 - val_loss: 0.7415 - val_accuracy: 0.5249
Epoch 290/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5926 - accuracy: 0.6445 - val_loss: 0.7574 - val_accuracy: 0.5083
Epoch 291/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6089 - accuracy: 0.6528 - val_loss: 0.7610 - val_accuracy: 0.5083
Epoch 292/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5902 - accuracy: 0.6694 - val_loss: 0.7354 - val_accuracy: 0.4972
Epoch 293/300
37/37 [==============================] - 0s 3ms/step - loss: 0.6068 - accuracy: 0.6445 - val_loss: 0.7404 - val_accuracy: 0.5193
Epoch 294/300

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              904 non-null    int64
 1   age                 904 non-null    int64
 2   workout_experience  904 non-null    int64
 3   workout_time        904 non-null    int64
 4   weight              904 non-null    int64
 5   height              904 non-null    int64
 6   fitness_goal        904 non-null    int64
dtypes: int64(7)
memory usage: 49.6 KB


In [62]:
input_data = {
    'gender' : 1,
    'age'    :35,
    'workout_experience' : 3,
    'workout_time' : 60,
    'weight' : 80,
    'height' : 170,
}

In [63]:
input_dict = {}

In [64]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [65]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([35])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([3])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([80])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>}

In [66]:
predictions = model.predict(input_dict)

1/1 [==============================] - 0s 33ms/step


In [67]:
predictions

array([[0.51008093]], dtype=float32)

In [68]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !
